<img src='ex12-2.png'>

In [80]:
import torch
import torch.nn as nn
import torch.optim as optim

In [81]:
one_hot_lookup = [
    [1, 0, 0, 0, 0], # 0 h
    [0, 1, 0, 0, 0], # 1 i
    [0, 0, 1, 0, 0], # 2 e
    [0, 0, 0, 1, 0], # 3 l
    [0, 0, 0, 0, 1], # 4 o
]
x_data = [0, 1, 0, 2, 3, 3] # hihell
y_data = [1, 0, 2, 3, 3, 4] # ihello
x_one_hot = [one_hot_lookup[i] for i in x_data]

# (2) Parameters

In [82]:
num_classes = 5
input_size = 5  # one_hot size
hidden_size = 5 # output from the LSTM. 5 to directly predict one-hot
batch_size = 1  # one sentence
sequence_length = 1 # Let's do one by one
num_layers = 1  # one-layer rnn

In [83]:
inputs = torch.tensor(x_one_hot, dtype=torch.float)
labels = torch.tensor(y_data, dtype=torch.long)

# 1. Model

In [84]:
class Model(nn.Module):
    def __init__(self,
                input_size=5,
                hidden_size=5,
                num_layers=1,
                batch_size=1,
                sequence_length=1,
                num_classes=5):
        super().__init__()
        self.rnn = nn.RNN(input_size=input_size,
                         hidden_size=hidden_size,
                         batch_first=True)
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.sequence_length = sequence_length
        self.num_classes = num_classes
        
        # Fully-Connected layer
        self.fc = nn.Linear(num_classes, num_classes)

    def forward(self, x, hidden):
        # Reshape input in (batch_size, sequence_length, input_size)
        x = x.view(self.batch_size, self.sequence_length, self.input_size)

        out, hidden = self.rnn(x, hidden)
        out = self.fc(out) # Add here
        out = out.view(-1, self.num_classes)
        return hidden, out
    
    def init_hidden(self):
        return torch.zeros(self.num_layers, self.batch_size, self.hidden_size)


# 2. Criterion & Loss

In [85]:
model = Model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

# 3. Training

In [86]:
model = Model(input_size=5, hidden_size=5, num_layers=1, batch_size=1, sequence_length=6, num_classes=5)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

In [87]:
hidden = model.init_hidden()
loss = 0

In [88]:
idx2char = ['h', 'i', 'e', 'l', 'o']
x_data = [0, 1, 0, 2, 3, 3] # hihell
one_hot_dict = {
    'h': [1, 0, 0, 0, 0],
    'i': [0, 1, 0, 0, 0],
    'e': [0, 0, 1, 0, 0],
    'l': [0, 0, 0, 1, 0],
    'o': [0, 0, 0, 0, 1],
}
one_hot_lookup = [
    [1, 0, 0, 0, 0], # 0 h
    [0, 1, 0, 0, 0], # 1 i
    [0, 0, 1, 0, 0], # 2 e
    [0, 0, 0, 1, 0], # 3 l
    [0, 0, 0, 0, 1], # 4 o
]
y_data = [1, 0, 2, 3, 3, 4] # ihello
x_one_hot = [one_hot_lookup[x] for x in x_data]

In [89]:
inputs = torch.tensor(x_one_hot, dtype=torch.float)
labels = torch.tensor(y_data, dtype=torch.long)

In [90]:
inputs

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.]])

In [91]:
labels

tensor([1, 0, 2, 3, 3, 4])

In [92]:
for epoch in range(0, 15 + 1):
    hidden.detach_()
    hidden = hidden.detach()
    hidden = hidden.clone().detach().requires_grad_(True) # New syntax from `1.0`
    
    hidden, outputs = model(inputs, hidden)
    optimizer.zero_grad()
    loss = criterion(outputs, labels) # It wraps for-loop in here
    loss.backward()
    optimizer.step()
    _, idx = outputs.max(1)
    idx = idx.data.numpy()
    result_str = [idx2char[c] for c in idx.squeeze()]
    print(f"epoch: {epoch}, loss: {loss.data}")
    print(f"Predicted string: {''.join(result_str)}")

epoch: 0, loss: 1.632482886314392
Predicted string: eieeee
epoch: 1, loss: 1.344533920288086
Predicted string: olello
epoch: 2, loss: 1.0991240739822388
Predicted string: olelll
epoch: 3, loss: 0.8392814993858337
Predicted string: ihello
epoch: 4, loss: 0.6179984211921692
Predicted string: ihello
epoch: 5, loss: 0.45398271083831787
Predicted string: ihello
epoch: 6, loss: 0.32671499252319336
Predicted string: ihello
epoch: 7, loss: 0.22967374324798584
Predicted string: ihello
epoch: 8, loss: 0.15975196659564972
Predicted string: ihello
epoch: 9, loss: 0.1100870743393898
Predicted string: ihello
epoch: 10, loss: 0.07598868757486343
Predicted string: ihello
epoch: 11, loss: 0.05339379981160164
Predicted string: ihello
epoch: 12, loss: 0.03852824494242668
Predicted string: ihello
epoch: 13, loss: 0.028578201308846474
Predicted string: ihello
epoch: 14, loss: 0.021733442321419716
Predicted string: ihello
epoch: 15, loss: 0.016893386840820312
Predicted string: ihello
